##part a) Neural network with numpy
here we are making a neural network with 3 inputs and 2 outputs. Relu function is used for non linearity at each layer. 2 hidden layers are used -one consisting of 5 neurons and second hidden layer consisting of four neurons.
We are trying to predict more than one value. we use dtwo non linear functions-

5x1^2+x2^2+x3^2+5x1+x2+x3
4x1^2+x2^2+2x3^2+4x1+x2+2x3




In [1]:
import numpy as np
import random
from numpy import array
from numpy.random import uniform
from numpy import hstack

In [2]:
#creating a class to define linear layers , here input dim represents number of input attributes, num_hidden is number of output neurons  
class Linear:
  def __init__(self,input_dim:int,num_hidden:int):
    self.weights=np.random.randn(input_dim,num_hidden)*np.sqrt(2. /input_dim)
    self.bias=np.zeros(num_hidden)

  def __call__(self, x):
    self.x = x
    output = x @ self.weights + self.bias
    #print(output.shape)
    return output

  def backward(self, gradient): # calculating loss function gradient
    self.weights_gradient = self.x.T @ gradient
    self.bias_gradient = gradient.sum(axis=0)
    
    self.x_gradient = gradient @ self.weights.T
    return self.x_gradient

  def update(self, lr): # updating weights and bias
    self.weights = self.weights - lr * self.weights_gradient
    self.bias = self.bias - lr * self.bias_gradient



In [3]:
#defining relu function and its derative
class Relu:
  def __call__(self,input_):
    self.input_=input_
    self.output=np.clip(self.input_,0,None)
    return self.output
  def backward(self,output_gradient):
    self.input_gradient=(self.input_>0)*output_gradient
    return self.input_gradient



In [4]:
#creating Non linear model consisting of 3 layers-2 hidden and one output layer consisting of two output neurons.
class Model:
  def __init__(self,input_dim:int,num_hidden1:int,num_hidden2:int):
    self.linear1=Linear(input_dim, num_hidden1)
    self.relu1=Relu()
    self.linear2=Linear(num_hidden1,num_hidden2)
    self.relu2=Relu()
    self.linear3=Linear(num_hidden2,2)
  
  def __call__(self,x): # forward propagation step
    l1=self.linear1(x)
    r1=self.relu1(l1)
    l2=self.linear2(r1)
    r2=self.relu2(l2)
    l3=self.linear3(r2)
    return l3

  def backward(self,output_gradient): # Backward Propagation step
    linear3_gradient=self.linear3.backward(output_gradient)
    relu_gradient1=self.relu2.backward(linear3_gradient)
    linear2_gradient=self.linear2.backward(relu_gradient1)
    relu_gradient2=self.relu1.backward(linear2_gradient)
    linear1_gradient=self.linear1.backward(relu_gradient2)
    return linear1_gradient
  
  def update(self,lr): # updating weights for each layer
    self.linear3.update(lr)
    self.linear2.update(lr)
    self.linear1.update(lr)




In [5]:
#defining loss function-mse and its derivative  
class MSE:
  def __call__(self,y_pred,y_true):
    self.y_pred=y_pred
    self.y_true=y_true
    return ((y_pred-y_true)**2).mean()
  def backward(self):
   n=self.y_true.shape[0]
   self.gradient=2.*(self.y_pred-self.y_true)/n
   return self.gradient

In [6]:
# running model for n number of epochs and try to compute loss function as minimum
from typing import Callable
def fit(x,y,model:Callable,loss:Callable,lr:float,num_epochs: int):
  for epoch in range(num_epochs):
    y_pred=model(x)
    loss_value=loss(y_pred,y)
    print(f"Epoch{epoch}, loss{loss_value}")
    gradient_from_loss=loss.backward()
    model.backward(gradient_from_loss)
    model.update(lr)



In [7]:
n=400  # number of input samples
d=3    # number of dimensions
num_hidden1=5 # number of neurons for first hidden layer
num_hidden2=4 # numbe rof neurons for second hidden layer
x=np.random.uniform(-1,1,(n,d)) # generating data of size=400 X 3

w1_true=np.array([[5,1,1],]).T # defining weight matrix for first output neuron
w2_true=np.array([[4,1,1],]).T # defining weight matrix for second output neuron
bias_true=np.array([1])

y1=(x**2) @ w1_true+x @ w1_true+bias_true #final layer output of first neuron 
y2=(x**2) @ w2_true+x @ w2_true+bias_true #final layer output of seconf layer neuron
y_true=hstack((y1, y2)) # joining output of first and second neuron
#lr = .0001
print(f'x: {x.shape},weights:{w1_true.shape}, bias: {bias_true.shape},y:{y_true.shape}')

loss=MSE() 
model=Model(d,num_hidden1,num_hidden2) 

fit(x,y_true, model=model, loss=loss, lr=0.01, num_epochs=50) # running the non linear model


x: (400, 3),weights:(3, 1), bias: (1,),y:(400, 2)
Epoch0, loss22.036907957639176
Epoch1, loss21.289515205582344
Epoch2, loss20.61674895130971
Epoch3, loss19.98946085404686
Epoch4, loss19.390641106636593
Epoch5, loss18.80680190289371
Epoch6, loss18.223301221371436
Epoch7, loss17.62898482702731
Epoch8, loss17.014805559971936
Epoch9, loss16.375858016205097
Epoch10, loss15.704245073679283
Epoch11, loss14.996096849260361
Epoch12, loss14.244610598391654
Epoch13, loss13.448858285400703
Epoch14, loss12.612470088900865
Epoch15, loss11.739088591523977
Epoch16, loss10.839710069435602
Epoch17, loss9.92766186605316
Epoch18, loss9.001271216341095
Epoch19, loss8.106966660089563
Epoch20, loss7.269850572221048
Epoch21, loss6.498871141675122
Epoch22, loss5.803046287934313
Epoch23, loss5.178831069368448
Epoch24, loss4.623852185058857
Epoch25, loss4.13116075383858
Epoch26, loss3.699187597065685
Epoch27, loss3.323246428842868
Epoch28, loss2.993828385301017
Epoch29, loss2.706738246399508
Epoch30, loss2.4581

In [8]:
def plot_intereactive_3d(x, y, y_pred=None):
  import plotly.graph_objects as go # Use plotly to create interactive graph

  fig = go.Figure()
  fig.add_trace(go.Scatter3d(x = x[:,0],
                    y = x[:,1],
                    z = y.reshape([-1]),
                    opacity=0.5, mode='markers', name='Underlying Function'
                    ))
 
  if y_pred is not None:
    fig.add_trace(go.Scatter3d(x = x[:,0],
                   y = x[:,1],
                   z = y_pred.reshape([-1]),
                   opacity=0.5, mode='markers', name='Predicted Function'
                  ))
    
  fig.update_layout(scene = dict(
                    xaxis_title='X1',
                    yaxis_title='X2',
                    zaxis_title='Y'),
                    width=700,
                    margin=dict(r=20, b=10, l=10, t=10))
  fig.show()

In [11]:
from sklearn.manifold import TSNE
X_red = TSNE(n_components=2).fit_transform(x)
y_true_red = TSNE(n_components=1).fit_transform(y_true)
y_pred_red = TSNE(n_components=1).fit_transform(model(x))
print(f'X_red: {X_red.shape}, y_true_red: {y_true_red.shape}, y_pred_red: {y_pred_red.shape}')
plot_intereactive_3d(X_red,y_true_red,y_pred_red)

X_red: (400, 2), y_true_red: (400, 1), y_pred_red: (400, 1)
